In [15]:
import numpy as np
from ct_support_code import *
import matplotlib.pyplot as plt

data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [16]:
# 1 a)
y_train_mean = y_train.mean()
print('y_train mean: ', y_train_mean)

y_val_mean = y_val.mean()
y_val_error = y_val.std()/np.sqrt(len(y_val))

y_train_error = y_train[:5785].std()/np.sqrt(5785)
print(f'y_val mean with standard error: {y_val_mean} ± {y_val_error}')
print(f'y_train 5785 samples mean with standard error: {y_train[:5785].mean()} ± {y_train_error}')

"""
We see that y_train mean with the error bar for
only 5785 samples is far away from zero (y_train mean).
As number of samples in y_train is much larger than number of samples in y_val,
we can't make our predictions of y_val mean just based on y_train mean 
with the standard error bar...better explanation..
"""

y_train mean:  -9.13868774539957e-15
y_val mean with standard error: -0.2160085093241599 ± 0.012903383410668334
y_train 5785 samples mean with standard error: -0.44247687859693674 ± 0.01192627246273395


"\nWe see that y_train mean with the error bar for\nonly 5785 samples is far away from zero (y_train mean).\nAs number of samples in y_train is much larger than number of samples in y_val,\nwe can't make our predictions of y_val mean just based on y_train mean \nwith the standard error bar...better explanation..\n"

In [17]:
# 1 b)

remove_indexes = [i for i,col in enumerate(X_train.T) if np.all(col==col[0])]
print('Indexes of constant columns:', remove_indexes)
remove_indexes_2 = []
for i, col1 in enumerate(X_train.T):
    for j, col2 in enumerate(X_train.T[i+1:,:]):
        if np.all(col1==col2):
            remove_indexes_2.append(i+j+1)
            break       
print('Original indexes of duplicate columns', remove_indexes_2)
remove_indexes = remove_indexes + remove_indexes_2
X_train = np.delete(X_train, remove_indexes, axis=1)
X_val = np.delete(X_val, remove_indexes, axis=1)
X_test = np.delete(X_test, remove_indexes, axis=1)

Indexes of constant columns: [59, 69, 179, 189, 351]
Original indexes of duplicate columns [78, 79, 69, 179, 199, 188, 189, 351, 287, 359]


In [18]:
# 2 
def fit_linreg(X, yy, alpha):
    K = X.shape[1]
    N = X.shape[0]
    X = np.column_stack((X,np.ones(N)))
    X = np.row_stack((X,np.sqrt(alpha)*np.eye(K+1)))
    y = np.concatenate((yy, np.zeros(K+1)))
    w = np.linalg.lstsq(X,y,rcond=None)[0]
    return w[:-1], w[-1]

def rmse(ww, bb, XX, yy):
    return np.sqrt(np.mean(((XX @ ww + bb) - yy)**2))

alpha = 30
train_w1, train_b1 = fit_linreg(X_train, y_train, alpha)
train_w2, train_b2 = fit_linreg_gradopt(X_train, y_train, alpha)
print('Training RMSE (fit_linreg): ', rmse(train_w1, train_b1, X_train, y_train))
print('Training RMSE (fit_linreg_gradopt): ', rmse(train_w2, train_b2, X_train, y_train))
print('Validation RMSE (fit_linreg): ', rmse(train_w1, train_b1, X_val, y_val))
print('Validation RMSE (fit_linreg_gradopt): ', rmse(train_w2, train_b2, X_val, y_val))

"""
Values close to each other, but not exactly the same
fit_linreg_gradopt is better on training
Add explanation..numerical reasons..different method?
val rmse smaller..?
"""

Training RMSE (fit_linreg):  0.3567669814948786
Training RMSE (fit_linreg_gradopt):  0.35675561493545876
Validation RMSE (fit_linreg):  0.42292954946321326
Validation RMSE (fit_linreg_gradopt):  0.42305590683687927


'\nValues close to each other, but not exactly the same\nfit_linreg_gradopt is better on training\nAdd explanation..numerical reasons..different method?\nval rmse smaller..?\n'

In [19]:
# 3
K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)


def fit_logreg_gradopt(X, yy, alpha):
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

def logreg_forward(X, ww, bb):
    aa = (X @ ww) + bb
    return 1 / (1 + np.exp(-aa))

N = X_train.shape[0]
D = X_train.shape[1]
WW = np.empty((D, K))
BB = np.empty(K)
LL = np.empty((N, K))
for kk in range(K):
    labels = y_train > thresholds[kk]
    LL[:, kk] = labels
    WW[:,kk], BB[kk] = fit_logreg_gradopt(X_train, labels, alpha)


X_train_logreg = logreg_forward(X_train, WW, BB) 
X_val_logreg = logreg_forward(X_val, WW, BB) 

train_w_lr, train_b_lr = fit_linreg_gradopt(X_train_logreg, y_train, alpha)
print('Training RMSE logreg transformation (fit_linreg_gradopt): ', 
      rmse(train_w_lr, train_b_lr, X_train_logreg, y_train))
print('Validation RMSE logreg transformation (fit_linreg_gradopt): ', 
      rmse(train_w_lr, train_b_lr, X_val_logreg, y_val))
""" Lower RMSE than normal linear regression """

Training RMSE logreg transformation (fit_linreg_gradopt):  0.15441194427307056
Validation RMSE logreg transformation (fit_linreg_gradopt):  0.2542485765839827


' Lower RMSE than normal linear regression '

In [26]:
# 4
def fit_nn_gradopt(X, yy, alpha, K=20, init=None):
    D = X.shape[1]
    args = (X, yy, alpha)
    if init==None:
        init = (np.random.randn(K), np.array(0), np.random.randn(K, D), np.random.randn(K))
    ww, bb, V, bk = minimize_list(nn_cost, init, args)
    return ww, bb, V, bk

def nn_rmse(params, XX, yy):
    ww, bb, V, bk = params
    
    A = np.dot(XX, V.T) + bk[None,:] # N,K
    P = 1 / (1 + np.exp(-A)) # N,K
    F = np.dot(P, ww) + bb # N,
    E =  np.sqrt(np.mean((F - yy)**2))
    return E

nn_rand_params = fit_nn_gradopt(X_train, y_train, 30)
q3_params = (train_w_lr, train_b_lr, WW.T, BB)
nn_q3_params = fit_nn_gradopt(X_train, y_train, 30, init=q3_params)
#print("NN cost with random initialization parameters: ", nn_cost(nn_rand_params, X_train, yy=y_train, alpha=30)[0])
#print("NN cost with q3 initialization parameters: ", nn_cost(nn_q3_params, X_train, yy=y_train, alpha=30)[0])
print("NN train RMSE with random initialization parameters: ", nn_rmse(nn_rand_params, X_train, y_train))
print("NN val RMSE with random initialization parameters: ", nn_rmse(nn_rand_params, X_val, y_val))
print("NN train RMSE with q3 initialization parameters: ", nn_rmse(nn_q3_params, X_train, y_train))
print("NN val RMSE with q3 initialization parameters: ", nn_rmse(nn_q3_params, X_val, y_val))

NN train RMSE with random initialization parameters:  0.13942065852907823
NN val RMSE with random initialization parameters:  0.2684920995102662
NN train RMSE with q3 initialization parameters:  0.13858869596445625
NN val RMSE with q3 initialization parameters:  0.2706637451683212


In [25]:
# 5

def train_nn_reg(X_train, y_train, X_val, y_val, alpha):
    params = fit_nn_gradopt(X_train, y_train, alpha)
    return nn_rmse(params, X_val, y_val)

def prob_imp(mu, cov, yy, alphas, alpha):
    a_idx = np.where(alphas==alpha)[0][0]
    #pi = norm.cdf((mu[a_idx]-np.max(yy))/np.sqrt(cov[a_idx,a_idx]))
    pi = (mu[a_idx] - np.max(yy)) / np.sqrt(cov[a_idx, a_idx])
    return pi

alphas = np.arange(0, 50, 0.02)
idx = np.random.randint(0,len(alphas),size=3)
y_train_gp = np.array([])
train_alphas = alphas[idx]
for alpha in train_alphas:
    y_train_gp = np.append(y_train_gp, -np.log(train_nn_reg(X_train, y_train, X_val, y_val, alpha)))
nn_rand_params = fit_nn_gradopt(X_train, y_train, 30)
baseline = np.log(nn_rmse(nn_rand_params, X_val, y_val))
#y_train_alpha = baseline + y_train_alpha
test_alphas = np.delete(alphas, idx)

for i in range(5):
    mu, cov = gp_post_par(test_alphas, train_alphas, y_train_gp)

    plt.plot(test_alphas, mu, '-k', linewidth=2)
    std = np.sqrt(np.diag(cov))
    plt.plot(test_alphas, mu + 2 * std, '--k', linewidth=2)
    plt.plot(test_alphas, mu - 2 * std, '--k', linewidth=2)
    plt.show()

    best_alpha = test_alphas[0]
    best_pi = - 1e100
    for alpha in test_alphas:
        pi = prob_imp(mu, cov, y_train_gp, test_alphas, alpha)
        if pi > best_pi:
            best_pi = pi
            best_alpha = alpha
    print(best_alpha, best_pi)
    train_alphas = np.append(train_alphas, best_alpha)
    test_alphas = np.delete(test_alphas, np.where(test_alphas==best_alpha))
    y_train_gp = np.append(y_train_gp, - np.log(train_nn_reg(X_train, y_train, X_val, y_val, best_alpha)))
    #y_train_alpha = np.append(y_train_alpha, baseline -np.log(train_nn_reg(X_train, y_train, X_val, y_val, best_alpha)))
best_alpha = train_alphas[np.argmax(y_train_gp)]

val_rmse = train_nn_reg(X_train, y_train, X_val, y_val, best_alpha)
test_rmse = train_nn_reg(X_train, y_train, X_test, y_test, best_alpha)
print(train_alphas)
print('Best alpha: ', best_alpha)
print('Val RMSE: ', val_rmse)
print('Test RMSE: ', test_rmse)

0.2706637451683212

In [31]:
#6


def nn2_cost(params, X, yy=None, alpha=None):
    """NN_COST simple neural network cost function and gradients, or predictions

           E, params_bar = nn_cost([ww, bb, V, bk], X, yy, alpha)
                    pred = nn_cost([ww, bb, V, bk], X)

     Cost function E can be minimized with minimize_list

     Inputs:
             params (ww, bb, V, bk), where:
                    --------------------------------
                        ww K,  hidden-output weights
                        bb     scalar output bias
                         V K,D hidden-input weights
                        bk K,  hidden biases
                    --------------------------------
                  X N,D input design matrix
                 yy N,  regression targets
              alpha     scalar regularization for weights

     Outputs:
                     E  sum of squares error
            params_bar  gradients wrt params, same format as params
     OR
               pred N,  predictions if only params and X are given as inputs
    """
    # Unpack parameters from list
    ww, bb, V, bk, V2 ,b2 = params

    # Forwards computation of cost
    A = np.dot(X, V.T) + bk[None,:] # N,K
    P = 1 / (1 + np.exp(-A)) # N,K
    B = np.dot(P, V2.T) + b2[None,:] # 
    P2 = 1 / (1 + np.exp(-B)) # N,K
    F = np.dot(P2, ww) + bb # N,
    if yy is None:
        # user wants prediction rather than training signal:
        return F
    res = F - yy # N,
    E = np.dot(res, res) + alpha*(np.sum(V*V) + np.dot(ww,ww)) # 1x1

    return E


def fit_nn2_gradopt(X, yy, alpha, K=64, H = 32, init=None):
    D = X.shape[1]
    args = (X, yy, alpha)
    if init==None:
        init = (np.random.randn(H), np.array(0), np.random.randn(K, D), np.random.randn(K),
                np.random.randn(H,K), np.random.randn(H))
    ww, bb, V, bk, V2, b2 = minimize_list(nn2_cost, init, args)
    return ww, bb, V, bk, V2, b2

def nn2_rmse(params, XX, yy):
    ww, bb, V, bk, V2, b2 = params
    
    A = np.dot(XX, V.T) + bk[None,:] # N,H
    P = 1 / (1 + np.exp(-A)) # N,H
    B = np.dot(P, V2.T) + b2[None,:] # 
    P2 = 1 / (1 + np.exp(-B)) # N,K
    F = np.dot(P2, ww) + bb # N,
    E =  np.sqrt(np.mean((F - yy)**2))
    return E
K =64
H = 32
# params = (np.random.randn(H), np.array(0), np.random.randn(K, D), np.random.randn(K),
#                 np.random.randn(H,K), np.random.randn(H))
# print(nn2_rmse(params,X_val, y_val))
nn_rand_params = fit_nn2_gradopt(X_train, y_train, 30)

#q3_params = (train_w_lr, train_b_lr, WW.T, BB)
#nn_q3_params = fit_nn2gradopt(X_train, y_train, 30, init=q3_params)
#print("NN cost with random initialization parameters: ", nn_cost(nn_rand_params, X_train, yy=y_train, alpha=30)[0])
#print("NN cost with q3 initialization parameters: ", nn_cost(nn_q3_params, X_train, yy=y_train, alpha=30)[0])
print("NN train RMSE with random initialization parameters: ", nn2_rmse(nn_rand_params, X_train, y_train))
print("NN val RMSE with random initialization parameters: ", nn2_rmse(nn_rand_params, X_val, y_val))
#print("NN train RMSE with q3 initialization parameters: ", nn2_rmse(nn_q3_params, X_train, y_train))
#print("NN val RMSE with q3 initialization parameters: ", nn2_rmse(nn_q3_params, X_val, y_val))


TypeError: cannot unpack non-iterable numpy.float64 object

In [ ]:
WW.shape